In [9]:
import pandas as pd
import pingouin as pg
import numpy as np

import sklearn
import scikit_posthocs as sp
import statsmodels.formula.api as sfa
import statsmodels.api as sa
import os
from pathlib import Path

import scipy
import neurokit2 as nk

import tqdm 


from collections import Counter
import matplotlib.pyplot as plt

In [22]:
lstPIds = []
path = "./Test Data/"
for filename in os.listdir(path):
    if filename.endswith(".csv"): 
        lstPIds.append(int(filename.split("-")[0].replace("ID", "")))
    else:
        continue
lstPIds = list(set(lstPIds))
print(lstPIds)

[1001, 3001, 2003, 2001]


In [26]:
lstAll = []
for pid in tqdm.tqdm(lstPIds):
    
    dfState = pd.read_csv(f"{path}ID{pid}-state.csv")

    dfStart = dfState[dfState.State == "start"].copy()
    dfEnd = dfState[dfState.State == "end"][["Time"]].iloc[:len(dfStart)]
    dfStart = dfStart.rename(columns={"Time":"TimeStart"})
    dfStart.TimeStart = dfStart.TimeStart #+ 60
    dfStart["TimeEnd"] = dfEnd.Time.values
    del dfStart["State"]

    dfStart["Duration"] = dfStart.TimeEnd - dfStart.TimeStart
    df = dfStart

    dfNBack = pd.read_csv(f"{path}ID{pid}-feedback.csv")
    dfNBack = dfNBack[dfNBack.Time != "Time"]
    correct= (dfNBack["IsCorrect"]==True).count()

    lstNBack = []
    for i, e in df.iterrows():
        dfX = dfNBack[(dfNBack.Time > e.TimeStart) & (dfNBack.Time < e.TimeEnd )]
        if (len(dfX) == 0):
            print("No Data!")
        lstNBack.append(dfX.IsCorrect.mean())
    df["IsCorrect"] = lstNBack

    # Read Flow File
    dfFlow = pd.read_csv(f"{path}ID{pid}-flow.csv")
    dfFlow["TimeNorm"] = dfFlow.Time-dfState.Time.iloc[0]

    # Read Visitor File
    dfLiams = pd.read_csv(f"{path}ID{pid}-visitor.csv")
    dfLiams = dfLiams.drop_duplicates(subset=['Name'])
    dfLiams["TimeNorm"] = dfLiams.Time-dfState.Time.iloc[0]

    # Calculate Accuracy
    lstLiamClickData = []
    for i, e in df.iterrows():
        dfX = dfFlow[(dfFlow.Time > e.TimeStart) & (dfFlow.Time < e.TimeEnd )]
        dfY = dfLiams[(dfLiams.Time > e.TimeStart) & (dfLiams.Time < e.TimeEnd )]
        dfY = dfY[dfY.Type == "down"]
        
        dfZ = pd.merge(dfX, dfY[["Name", "Type"]], on="Name", how="outer")
        if ((len(dfX) == 0) | (len(dfY) == 0)):
            print("No Data! " + str(i))
           
   #    trueNegatives = len(dfZ[(dfZ.HasTicket == True) & (dfZ.Type.isna())])
   #     falseNegatives = len(dfZ[(dfZ.HasTicket == False) & (dfZ.Type.isna())])
   #     truePositive = len(dfZ[(dfZ.HasTicket == False) & (dfZ.Type == "down")]) # hits
   #     falsePositive = len(dfZ[(dfZ.HasTicket == True) & (dfZ.Type == "down")]) # CLicked but has ticked
   #     total = len(dfX)
        
   #     lstLiamClickData.append([total, truePositive, falsePositive, trueNegatives, falseNegatives])
   # lstLiamClickData = np.array(lstLiamClickData)
  #  df["LiamsCountTotal"] = lstLiamClickData[:,0]
  #  df["LiamsCounTruePositive"] = lstLiamClickData[:,1]
  #  df["LiamsCountFalsePositive"] = lstLiamClickData[:,2]
  #  df["LiamsCountTrueNegatives"] = lstLiamClickData[:,3]
  #  df["LiamsCountFalseNegatives"] = lstLiamClickData[:,4]

   # df["Accuracy"] = (df.LiamsCounTruePositive +  df.LiamsCountTrueNegatives) / df.LiamsCountTotal
   # df["Flow"] = df["LiamsCountTotal"] / df.Duration * 60
    # Read EDA
    dfEDA = pd.read_csv(f"{path}ID{pid}-EDA.csv")
    edaFs = 250  # Sampling frequency

    dfEDA.Value = (dfEDA.Value / 1000) / 25
    dfEDA["TimeNorm"] = dfEDA.Time-dfState.Time.iloc[0]
    dfEDA["EDAMedian"] = scipy.signal.medfilt(dfEDA.Value, edaFs +1)
    sos = scipy.signal.butter(1, 0.01, btype='lowpass', analog=False, output='sos', fs=edaFs)
    dfEDA["EDAButter"] = scipy.signal.sosfilt(sos, dfEDA["EDAMedian"])
    signal, info=nk.eda_process(nk.standardize(dfEDA.Value, sampling_rate=edaFs), sampling_rate=edaFs)
    dfEDA["EDARaw"] = signal.EDA_Raw
    dfEDA["EDATonic"] = signal.EDA_Tonic
    
    lstEDATonic = []
    lstEDARaw = []
    lstEDAMedian = []
    lstEDAButter = []
    for i, e in df.iterrows():
        dfX = dfEDA[(dfEDA.Time > e.TimeStart) & (dfEDA.Time < e.TimeEnd )]
        lstEDATonic.append(dfX.EDATonic.mean())
        lstEDARaw.append(dfX.EDARaw.mean())
        lstEDAMedian.append(dfX.EDAMedian.mean())
        lstEDAButter.append(dfX.EDAButter.mean())

    df["EDATonic"] = lstEDATonic
    df["EDARaw"] = lstEDARaw
    df["EDAMedian"] = lstEDAMedian
    df["EDAButter"] = lstEDAButter
    
    df["PId"] = pid
    lstAll.append(df)
    
dfAll = pd.concat(lstAll)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

No Data!
No Data! 0
No Data! 2
No Data! 4
No Data! 6
No Data! 8
No Data! 11


 25%|█████████████████████                                                               | 1/4 [00:21<01:03, 21.13s/it]

No Data!
No Data! 0
No Data! 2
No Data! 4
No Data! 6
No Data! 8
No Data! 11


 50%|██████████████████████████████████████████                                          | 2/4 [00:52<00:54, 27.44s/it]

No Data!
No Data! 0
No Data! 2
No Data! 4
No Data! 6
No Data! 8
No Data! 11


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [01:20<00:27, 27.34s/it]

No Data!
No Data! 0
No Data! 2
No Data! 4
No Data! 6
No Data! 8
No Data! 11


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:44<00:00, 26.19s/it]


In [27]:
dfAll

,TimeStart,BlockNumber,AdaptationStatus,NBackN,TimeEnd,Duration,IsCorrect,EDATonic,EDARaw,EDAMedian,EDAButter,PId
0,1.660049e+09,4,1,1,1.660050e+09,480.00000,0.945161,0.183294,0.183091,1.588189,1.553392,1001
2,1.660050e+09,2,1,1,1.660051e+09,480.00000,0.960894,-0.115683,-0.115613,1.516970,1.528111,1001
4,1.660051e+09,3,1,1,1.660051e+09,480.00000,0.962366,-0.189037,-0.189134,1.499480,1.511626,1001
6,1.660052e+09,1,1,1,1.660052e+09,480.00000,0.941748,0.191787,0.191968,1.590318,1.614060,1001
8,1.660052e+09,5,2,1,1.660053e+09,480.00000,0.966006,-0.750345,-0.750421,1.365699,1.370513,1001
11,1.660053e+09,-2,1,1,1.660053e+09,0.00000,NaN,NaN,NaN,NaN,NaN,1001
0,1.660119e+09,4,1,3,1.660120e+09,480.00000,0.836735,-1.482098,-1.482186,4.707537,4.619852,3001
2,1.660120e+09,3,1,3,1.660121e+09,480.00000,0.805000,0.135351,0.135721,6.999275,6.976391,3001
4,1.660121e+09,1,1,3,1.660121e+09,480.00000,0.823529,0.521130,0.521031,7.545083,7.560103,3001
6,1.660122e+09,5,2,3,1.660122e+09,480.00000,0.816754,0.163188,0.163666,7.038746,7.052064,3001


In [28]:
lstAll = []
for pid in tqdm.tqdm(lstPIds):
    
    dfState = pd.read_csv(f"{path}ID{pid}-state.csv")

    dfStart = dfState[dfState.State == "start"].copy()
    dfEnd = dfState[dfState.State == "end"][["Time"]].iloc[:len(dfStart)]
    dfStart = dfStart.rename(columns={"Time":"TimeStart"})
    dfStart.TimeStart = dfStart.TimeStart #+ 60
    dfStart["TimeEnd"] = dfEnd.Time.values
    del dfStart["State"]
    dfStart["Duration"] = dfStart.TimeEnd - dfStart.TimeStart
    df = dfStart[dfStart.BlockNumber != -2].copy()

    dfNBack = pd.read_csv(f"{path}ID{pid}-feedback.csv")
    dfNBack = dfNBack[dfNBack.Time != "Time"]
    correct= (dfNBack["IsCorrect"]==True).count()

    lstNBack = []
    for i, e in df.iterrows():
        dfX = dfNBack[(dfNBack.Time > e.TimeStart) & (dfNBack.Time < e.TimeEnd )]
        if (len(dfX) == 0):
            print("No Data!")
        lstNBack.append(dfX.IsCorrect.mean())
    df["IsCorrect"] = lstNBack

    lstCorrectCount = []
    lstTotalTouchCount = []
    #lstNBack = []
    for i, e in df.iterrows():
        dfX = dfNBack[(dfNBack.Time > e.TimeStart) & (dfNBack.Time <= e.TimeEnd )]
        lstCorrectCount.append(len(dfX[dfX.IsCorrect == True]))
        lstTotalTouchCount.append(len(dfX))
    
    dfSphere = pd.read_csv(f"{path}ID{pid}-sphere.csv")
    del dfSphere["Feedback"]
    
    lstMissedCount = []
    for i, e in df.iterrows():
        dfX = dfSphere[(dfSphere.Time > e.TimeStart) & (dfSphere.Time <= e.TimeEnd )].reset_index(drop=True)
        missed = 0
        for i in range(0, len(dfX)-1):
            if ((dfX.iloc[i].Type == "up") & (dfX.iloc[i+1].Type == "down")):
                if(dfX.iloc[i+1].Time-dfX.iloc[i].Time > 4):
                    missed += 1
        lstMissedCount.append(missed)
    
    df["NBackCorrect"] = lstCorrectCount
    df["NBackMissed"] = lstMissedCount
    df["NBackTotal"] = np.array(lstTotalTouchCount) + np.array(lstMissedCount)
    df["NBackAccuracy"] = df["NBackCorrect"]/df["NBackTotal"] * 100
    
    df["NBackTimePerBall"] = df["Duration"] / np.array(lstTotalTouchCount)
    
    
    # Read Flow File
    dfFlow = pd.read_csv(f"{path}ID{pid}-flow.csv")
    dfFlow["TimeNorm"] = dfFlow.Time-dfState.Time.iloc[0]

    # Read Visitor File
    #dfLiams = pd.read_csv(f"{path}ID{pid}-visitor.csv")
    #dfLiams = dfLiams.drop_duplicates(subset=['Name'])
    #dfLiams["TimeNorm"] = dfLiams.Time-dfState.Time.iloc[0]

   ## Calculate Accuracy
    lstLiamData = []
    for i, e in df.iterrows():
        dfX = dfFlow[(dfFlow.Time > e.TimeStart) & (dfFlow.Time < e.TimeEnd )]
       
        if ((len(dfX) == 0)):
            print("No Data! Liams" + str(i))
           
        lstLiamData.append(len(dfX))
        
    #lstLiamClickData.append([total, truePositive, falsePositive, trueNegatives, falseNegatives])
    #lstLiamClickData = np.array(lstLiamClickData)
    df["LiamsCountTotal"] = lstLiamData
    #df["LiamsCounTruePositive"] = lstLiamClickData[:,1]
    #df["LiamsCountFalsePositive"] = lstLiamClickData[:,2]
    #df["LiamsCountTrueNegatives"] = lstLiamClickData[:,3]
    #df["LiamsCountFalseNegatives"] = lstLiamClickData[:,4]

   # df["Accuracy"] = (df.LiamsCounTruePositive +  df.LiamsCountTrueNegatives) / df.LiamsCountTotal
    df["Flow"] = df["LiamsCountTotal"] / df.Duration * 60
    # Read EDA
    dfEDA = pd.read_csv(f"{path}ID{pid}-EDA.csv")
    edaFs = 250  # Sampling frequency

    dfEDA.Value = (dfEDA.Value / 1000) / 25
    dfEDA["TimeNorm"] = dfEDA.Time-dfState.Time.iloc[0]
    dfEDA["EDAClean"] =  nk.eda_clean(dfEDA["Value"], sampling_rate=edaFs) #scipy.signal.medfilt(dfEDA.Value, edaFs +1)
    #sos = scipy.signal.butter(1, 0.01, btype='lowpass', analog=False, output='sos', fs=edaFs)
    #dfEDA["EDAButter"] = scipy.signal.sosfilt(sos, dfEDA["EDAMedian"])
    #signal, info=nk.eda_process(nk.standardize(dfEDA.EDAClean, sampling_rate=edaFs), sampling_rate=edaFs)
    #signal2, info = nk.eda_sympathetic(dfEDA.EDAClean, sampling_rate=edaFs, frequency_band=[0.045, 0.25], method='posada', show=False)
    signal, info=nk.eda_process(dfEDA.EDAClean, sampling_rate=edaFs)
    dfEDA["EDARaw"] = signal.EDA_Raw
    dfEDA["EDATonic"] = signal.EDA_Tonic
    dfEDA["EDAPhasic"] = signal.EDA_Phasic
    #dfEDA["EDASCR"] = signal.SCR_Amplitude
    #dfEDA["EDASymp"] = signal2.EDA_Symp
    
    
    lstEDATonic = []
    lstEDARaw = []
    lstEDAMedian = []
    lstEDAButter = []
    lstEDAPhasic = []
    #lstEDASCR = []
    #lstEDAFre = []
    
    for i, e in df.iterrows():
        dfX = dfEDA[(dfEDA.Time > e.TimeStart) & (dfEDA.Time < e.TimeEnd )]
        lstEDATonic.append(dfX.EDATonic.mean())
        lstEDARaw.append(dfX.EDARaw.mean())
        lstEDAPhasic.append(dfX.EDAPhasic.mean())
        #lstEDASCR.append(dfX.EDASCR.mean())
        #lstEDAFre.append(dfX.EDAFre.mean())
        #lstEDAMedian.append(dfX.EDAClean.mean())
        #lstEDAButter.append(dfX.EDAButter.mean())

    df["EDATonic"] = lstEDATonic
    df["EDARaw"] = lstEDARaw
    df["EDAPhasic"] = lstEDAPhasic
    #df["EDASCR"] = lstEDASCR
    #df["EDAClean"] = lstEDAClean
    #df["EDAButter"] = lstEDAButter
   # dfEDA["EDASymp"] = lstEDASymp
    df["PId"] = pid
    lstAll.append(df)
    
dfAll = pd.concat(lstAll)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:12<00:00,  3.05s/it]


In [29]:
dfAll 

,TimeStart,BlockNumber,AdaptationStatus,NBackN,TimeEnd,Duration,IsCorrect,NBackCorrect,NBackMissed,NBackTotal,NBackAccuracy,NBackTimePerBall,LiamsCountTotal,Flow,EDATonic,EDARaw,EDAPhasic,PId
0,1.660049e+09,4,1,1,1.660050e+09,480.00000,0.945161,293,0,310,94.516129,1.548387,905,113.125000,1.588281,1.588233,-0.000048,1001
2,1.660050e+09,2,1,1,1.660051e+09,480.00000,0.960894,344,0,358,96.089385,1.340782,707,88.375000,1.517022,1.517038,0.000017,1001
4,1.660051e+09,3,1,1,1.660051e+09,480.00000,0.962366,358,0,372,96.236559,1.290323,1175,146.875000,1.499538,1.499515,-0.000023,1001
6,1.660052e+09,1,1,1,1.660052e+09,480.00000,0.941748,291,1,310,93.870968,1.553398,198,24.750000,1.590306,1.590349,0.000043,1001
8,1.660052e+09,5,2,1,1.660053e+09,480.00000,0.966006,341,1,354,96.327684,1.359773,861,107.625000,1.365752,1.365735,-0.000023,1001
0,1.660119e+09,4,1,3,1.660120e+09,480.00000,0.836735,164,2,198,82.828283,2.448980,857,107.125000,4.707897,4.707772,-0.000125,3001
2,1.660120e+09,3,1,3,1.660121e+09,480.00000,0.805000,161,2,202,79.702970,2.400000,1193,149.125000,6.998952,6.999476,0.000524,3001
4,1.660121e+09,1,1,3,1.660121e+09,480.00000,0.823529,154,0,187,82.352941,2.566845,196,24.500000,7.545393,7.545254,-0.000139,3001
6,1.660122e+09,5,2,3,1.660122e+09,480.00000,0.816754,156,4,195,80.000000,2.513089,908,113.500000,7.038382,7.039059,0.000677,3001
8,1.660122e+09,2,1,3,1.660123e+09,480.00000,0.860215,160,4,190,84.210526,2.580645,700,87.500000,6.305618,6.305830,0.002622,3001


In [31]:
dfAll = dfAll[dfAll.BlockNumber != -2]
dfAll

,TimeStart,BlockNumber,AdaptationStatus,NBackN,TimeEnd,Duration,IsCorrect,NBackCorrect,NBackMissed,NBackTotal,NBackAccuracy,NBackTimePerBall,LiamsCountTotal,Flow,EDATonic,EDARaw,EDAPhasic,PId
0,1.660049e+09,4,1,1,1.660050e+09,480.00000,0.945161,293,0,310,94.516129,1.548387,905,113.125000,1.588281,1.588233,-0.000048,1001
2,1.660050e+09,2,1,1,1.660051e+09,480.00000,0.960894,344,0,358,96.089385,1.340782,707,88.375000,1.517022,1.517038,0.000017,1001
4,1.660051e+09,3,1,1,1.660051e+09,480.00000,0.962366,358,0,372,96.236559,1.290323,1175,146.875000,1.499538,1.499515,-0.000023,1001
6,1.660052e+09,1,1,1,1.660052e+09,480.00000,0.941748,291,1,310,93.870968,1.553398,198,24.750000,1.590306,1.590349,0.000043,1001
8,1.660052e+09,5,2,1,1.660053e+09,480.00000,0.966006,341,1,354,96.327684,1.359773,861,107.625000,1.365752,1.365735,-0.000023,1001
0,1.660119e+09,4,1,3,1.660120e+09,480.00000,0.836735,164,2,198,82.828283,2.448980,857,107.125000,4.707897,4.707772,-0.000125,3001
2,1.660120e+09,3,1,3,1.660121e+09,480.00000,0.805000,161,2,202,79.702970,2.400000,1193,149.125000,6.998952,6.999476,0.000524,3001
4,1.660121e+09,1,1,3,1.660121e+09,480.00000,0.823529,154,0,187,82.352941,2.566845,196,24.500000,7.545393,7.545254,-0.000139,3001
6,1.660122e+09,5,2,3,1.660122e+09,480.00000,0.816754,156,4,195,80.000000,2.513089,908,113.500000,7.038382,7.039059,0.000677,3001
8,1.660122e+09,2,1,3,1.660123e+09,480.00000,0.860215,160,4,190,84.210526,2.580645,700,87.500000,6.305618,6.305830,0.002622,3001


,TimeStart,BlockNumber,AdaptationStatus,TimeEnd,Duration,IsCorrect,NBackCorrect,NBackMissed,NBackTotal,NBackAccuracy,NBackTimePerBall,LiamsCountTotal,Flow,EDATonic,EDARaw,EDAPhasic,PId,BackLevel
0,1.660049e+09,4,1,1.660050e+09,420.02791,0.942857,264,0,280,94.285714,1.500100,819,116.992226,1.673971,1.673948,-0.000022,1001,1
2,1.660050e+09,2,1,1.660051e+09,420.01837,0.964630,300,0,311,96.463023,1.350541,616,87.996151,1.489366,1.489371,0.000005,1001,1
4,1.660051e+09,3,1,1.660051e+09,420.03304,0.959248,306,0,319,95.924765,1.316718,1033,147.559821,1.466732,1.466726,-0.000005,1001,1
6,1.660052e+09,1,1,1.660052e+09,420.02532,0.941392,257,1,274,93.795620,1.538554,169,24.141402,1.530425,1.530437,0.000012,1001,1
8,1.660052e+09,5,2,1.660053e+09,420.04154,0.966997,293,1,304,96.381579,1.386276,779,111.274709,1.348287,1.348295,0.000002,1001,1
0,1.660119e+09,4,1,1.660120e+09,420.02638,0.835165,152,0,182,83.516484,2.307837,757,108.136065,4.843129,4.842205,-0.000923,3001,3
2,1.660120e+09,3,1,1.660121e+09,420.01750,0.813187,148,2,184,80.434783,2.307788,1040,148.565238,7.030451,7.030824,0.000373,3001,3
4,1.660121e+09,1,1,1.660121e+09,420.03071,0.810976,133,0,164,81.097561,2.561163,172,24.569632,7.483473,7.483040,-0.000432,3001,3
6,1.660122e+09,5,2,1.660122e+09,420.03020,0.836257,143,3,174,82.183908,2.456317,832,118.848597,7.049892,7.049674,-0.000218,3001,3
8,1.660122e+09,2,1,1.660123e+09,420.02659,0.855422,142,3,169,84.023669,2.530281,614,87.708733,6.279494,6.278323,0.001646,3001,3


In [18]:
dfAll.groupby("BlockNumber").Flow.mean()

BlockNumber
1     24.665291
2     87.472002
3    148.275614
4     99.898250
5    103.135048
Name: Flow, dtype: float64

In [32]:
dfAll.to_csv("adaptivedf.csv", index=False)
dfAll

,TimeStart,BlockNumber,AdaptationStatus,NBackN,TimeEnd,Duration,IsCorrect,NBackCorrect,NBackMissed,NBackTotal,NBackAccuracy,NBackTimePerBall,LiamsCountTotal,Flow,EDATonic,EDARaw,EDAPhasic,PId
0,1.660049e+09,4,1,1,1.660050e+09,480.00000,0.945161,293,0,310,94.516129,1.548387,905,113.125000,1.588281,1.588233,-0.000048,1001
2,1.660050e+09,2,1,1,1.660051e+09,480.00000,0.960894,344,0,358,96.089385,1.340782,707,88.375000,1.517022,1.517038,0.000017,1001
4,1.660051e+09,3,1,1,1.660051e+09,480.00000,0.962366,358,0,372,96.236559,1.290323,1175,146.875000,1.499538,1.499515,-0.000023,1001
6,1.660052e+09,1,1,1,1.660052e+09,480.00000,0.941748,291,1,310,93.870968,1.553398,198,24.750000,1.590306,1.590349,0.000043,1001
8,1.660052e+09,5,2,1,1.660053e+09,480.00000,0.966006,341,1,354,96.327684,1.359773,861,107.625000,1.365752,1.365735,-0.000023,1001
0,1.660119e+09,4,1,3,1.660120e+09,480.00000,0.836735,164,2,198,82.828283,2.448980,857,107.125000,4.707897,4.707772,-0.000125,3001
2,1.660120e+09,3,1,3,1.660121e+09,480.00000,0.805000,161,2,202,79.702970,2.400000,1193,149.125000,6.998952,6.999476,0.000524,3001
4,1.660121e+09,1,1,3,1.660121e+09,480.00000,0.823529,154,0,187,82.352941,2.566845,196,24.500000,7.545393,7.545254,-0.000139,3001
6,1.660122e+09,5,2,3,1.660122e+09,480.00000,0.816754,156,4,195,80.000000,2.513089,908,113.500000,7.038382,7.039059,0.000677,3001
8,1.660122e+09,2,1,3,1.660123e+09,480.00000,0.860215,160,4,190,84.210526,2.580645,700,87.500000,6.305618,6.305830,0.002622,3001
